In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.insert(0, '/Users/florianrunkel/Documents/02_Uni/04_Masterarbeit/masterthesis/')

from backend.ml_pipe.data.database.mongodb import MongoDb

# Matplotlib Einstellungen für deutsche Beschriftungen
plt.rcParams['font.size'] = 10
plt.style.use('default')

In [2]:
try:
    mongo_client = MongoDb()

    result = mongo_client.get_all('feedback')
    feedback_data = result.get('data', [])

    print(feedback_data)

except Exception as e:
    print(f"Error connecting to MongoDB: {e}")

[{'_id': '68c8349bcd07e1b6a3665a13', 'uid': 'UID001', 'freeText': '', 'prognoseBewertung': [{'modell': '', 'prognose': '', 'echt': '', 'bemerkung': ''}], 'bewertungsskala': [4, 4, 3, 4, 5, 5, 5, 4, 5, 3, 5], 'explanationFeedback': {}, 'timestamp': '2025-09-15T15:45:31.111053Z'}, {'_id': '68c834ea4c8877fd37754728', 'uid': 'UID001', 'freeText': '', 'prognoseBewertung': [{'modell': '', 'prognose': '', 'echt': '', 'bemerkung': ''}], 'bewertungsskala': [3, 5, 4, 4, 3, 5, 5, 4, 4, 3, 5], 'explanationFeedback': {}, 'timestamp': '2025-09-15T15:46:50.816364Z'}, {'_id': '68c8363e4c8877fd3775472a', 'uid': 'UID001', 'freeText': '', 'prognoseBewertung': [{'modell': '', 'prognose': '', 'echt': '', 'bemerkung': ''}], 'bewertungsskala': [4, 5, 5, 4, 4, 5, 5, 4, 3, 5, 5], 'explanationFeedback': {}, 'timestamp': '2025-09-15T15:52:29.992306Z'}, {'_id': '68c8373dcd07e1b6a3665a15', 'uid': 'UID001', 'freeText': '', 'prognoseBewertung': [{'modell': '', 'prognose': '', 'echt': '', 'bemerkung': ''}], 'bewertun

In [3]:
import os
os.environ["PYMONGO_DISABLE_IPV6"] = "1"     # IPv6 erstmal ausschalten
os.environ["DNSPYTHON_IPV6"] = "false"

import dns.resolver
r = dns.resolver.Resolver(configure=True)
r.nameservers = ["1.1.1.1", "8.8.8.8"]       # Router-DNS umgehen
r.timeout = 2.0
r.lifetime = 3.0
dns.resolver.default_resolver = r            # global setzen

# jetzt erst Deinen Code:
mongo_client = MongoDb()
result = mongo_client.get_all("feedback")
print(result)

{'statusCode': 200, 'data': [{'_id': '68c8349bcd07e1b6a3665a13', 'uid': 'UID001', 'freeText': '', 'prognoseBewertung': [{'modell': '', 'prognose': '', 'echt': '', 'bemerkung': ''}], 'bewertungsskala': [4, 4, 3, 4, 5, 5, 5, 4, 5, 3, 5], 'explanationFeedback': {}, 'timestamp': '2025-09-15T15:45:31.111053Z'}, {'_id': '68c834ea4c8877fd37754728', 'uid': 'UID001', 'freeText': '', 'prognoseBewertung': [{'modell': '', 'prognose': '', 'echt': '', 'bemerkung': ''}], 'bewertungsskala': [3, 5, 4, 4, 3, 5, 5, 4, 4, 3, 5], 'explanationFeedback': {}, 'timestamp': '2025-09-15T15:46:50.816364Z'}, {'_id': '68c8363e4c8877fd3775472a', 'uid': 'UID001', 'freeText': '', 'prognoseBewertung': [{'modell': '', 'prognose': '', 'echt': '', 'bemerkung': ''}], 'bewertungsskala': [4, 5, 5, 4, 4, 5, 5, 4, 3, 5, 5], 'explanationFeedback': {}, 'timestamp': '2025-09-15T15:52:29.992306Z'}, {'_id': '68c8373dcd07e1b6a3665a15', 'uid': 'UID001', 'freeText': '', 'prognoseBewertung': [{'modell': '', 'prognose': '', 'echt': '', 

In [4]:
print("\nDATENVERARBEITUNG")
print("=" * 30)

if result and 'data' in result:
    feedback_data = result['data']
    print(f"{len(feedback_data)} Feedback-Einträge geladen")
else:
    print("Keine Daten verfügbar")
    feedback_data = []

if len(feedback_data) > 0:
    df = pd.DataFrame(feedback_data)

    df['timestamp'] = pd.to_datetime(df['timestamp'])

    df['rating_length'] = df['bewertungsskala'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    print(f"\nBewertungsskala-Längen:")
    print(df['rating_length'].value_counts().sort_index())

    df['group'] = df['rating_length'].apply(lambda x: 'Experimental' if x == 11 else 'Control' if x == 8 else 'Unknown')

    for i in range(11):
        df[f'rating_{i+1}'] = df.apply(lambda row: 
            row['bewertungsskala'][i] if isinstance(row['bewertungsskala'], list) and len(row['bewertungsskala']) > i else np.nan, axis=1)

    control_df = df[df['group'] == 'Control'].copy()
    experimental_df = df[df['group'] == 'Experimental'].copy()

    # Grundlegende Statistiken
    print("\nFEEDBACK-DATA OVERVIEW")
    print("=" * 50)
    print(f"Total number of feedback: {len(df)}")  
    print(f"Control Group (8 Fragen): {len(control_df)} Teilnehmer")
    print(f"Experimental Group (11 Fragen): {len(experimental_df)} Teilnehmer")
    print(f"Unknown Group: {len(df[df['group'] == 'Unknown'])} Teilnehmer")

    # Zeige erste paar Zeilen
    print(f"\nERSTE 3 ZEILEN:")
    print(df[['uid', 'group', 'rating_length', 'timestamp']].head(3).to_string())
else:
    print("Keine Daten zum Verarbeiten verfügbar")



DATENVERARBEITUNG
11 Feedback-Einträge geladen

Bewertungsskala-Längen:
rating_length
8      1
11    10
Name: count, dtype: int64

FEEDBACK-DATA OVERVIEW
Total number of feedback: 11
Control Group (8 Fragen): 1 Teilnehmer
Experimental Group (11 Fragen): 10 Teilnehmer
Unknown Group: 0 Teilnehmer

ERSTE 3 ZEILEN:
      uid         group  rating_length                        timestamp
0  UID001  Experimental             11 2025-09-15 15:45:31.111053+00:00
1  UID001  Experimental             11 2025-09-15 15:46:50.816364+00:00
2  UID001  Experimental             11 2025-09-15 15:52:29.992306+00:00


In [5]:
# Control Group Questions (8 Fragen)
control_group_questions = [
    'The system\'s predictions about candidate job-switching readiness seemed realistic.',
    'The predictions were relevant for prioritizing candidates in Active Sourcing.',
    'I trusted the system\'s predictions when deciding which candidates to approach.',
    'The recommendations gave me enough confidence to base sourcing decisions on them.',
    'The system\'s predictions were easy to interpret without further explanation.',
    'The predictions helped me to structure the candidate selection process more efficiently.',
    'I can imagine using such a prediction system in my daily recruiting activities.',
    'The system would help me to improve the effectiveness of my sourcing decisions.'
]

# Experimental Group Questions (11 Fragen)
experimental_group_questions = [
    'The explanations made it clear why a candidate was predicted as more or less likely to switch jobs.',
    'The explanations increased my understanding of how the system generated its predictions.',
    'The explanations were concrete and applicable to my sourcing decisions.',
    'The explanations strengthened my confidence in the reliability of the predictions.',
    'The presence of explanations made me more willing to act on the system\'s recommendations.',
    'The combination of predictions and explanations was straightforward and clear to understand.',
    'The explanations improved my ability to identify which candidates should be prioritized in Active Sourcing.',
    'The explanations supported me in combining the system\'s predictions with my own recruiting expertise.',
    'The system complemented my judgment rather than replacing it.',
    'I could imagine integrating such a system with explanations into my daily recruiting workflow.',
    'The explanations provided added value compared to predictions alone.'
]

# Kategorien-Mapping für vergleichbare Fragen
comparable_questions = {
    'Relevance & Realism': {
        'control': [1, 2],  # Q1, Q2
        'experimental': [1, 2]  # Q1, Q2
    },
    'Confidence in Predictions': {
        'control': [3, 4],  # Q3, Q4
        'experimental': [4, 5]  # Q4, Q5
    },
    'Usability for Recruiting': {
        'control': [5, 6],  # Q5, Q6
        'experimental': [6, 7]  # Q6, Q7
    },
    'Perceived Value & Intention to Use': {
        'control': [7, 8],  # Q7, Q8
        'experimental': [10, 11]  # Q10, Q11
    }
}

# Statistische Tests für alle vergleichbaren Kategorien
print("\nSTATISTISCHE VERGLEICHE ZWISCHEN GRUPPEN:")
print("-" * 50)

from scipy.stats import mannwhitneyu
import numpy as np

comparison_results = []

for category, question_indices in comparable_questions.items():
    print(f"\n{category.upper()}:")
    print("-" * len(category))
    
    # Control Group Daten sammeln
    control_ratings = []
    for idx in question_indices['control']:
        col = f'rating_{idx}'
        if col in control_df.columns:
            ratings = control_df[col].dropna()
            control_ratings.extend(ratings.tolist())
    
    # Experimental Group Daten sammeln
    experimental_ratings = []
    for idx in question_indices['experimental']:
        col = f'rating_{idx}'
        if col in experimental_df.columns:
            ratings = experimental_df[col].dropna()
            experimental_ratings.extend(ratings.tolist())
    
    if len(control_ratings) > 0 and len(experimental_ratings) > 0:
        # Mann-Whitney-U Test
        stat, p_value = mannwhitneyu(control_ratings, experimental_ratings, alternative='two-sided')
        
        # Effektstärke (Cohen's d)
        n1, n2 = len(control_ratings), len(experimental_ratings)
        pooled_std = np.sqrt(((n1-1)*np.var(control_ratings) + (n2-1)*np.var(experimental_ratings)) / (n1+n2-2))
        cohens_d = (np.mean(control_ratings) - np.mean(experimental_ratings)) / pooled_std if pooled_std > 0 else 0
        
        # Ergebnisse speichern
        result = {
            'category': category,
            'control_mean': np.mean(control_ratings),
            'experimental_mean': np.mean(experimental_ratings),
            'control_n': len(control_ratings),
            'experimental_n': len(experimental_ratings),
            'u_statistic': stat,
            'p_value': p_value,
            'cohens_d': cohens_d,
            'significant': p_value < 0.05
        }
        comparison_results.append(result)
        
        # Ausgabe
        print(f"  Control Group:    M = {result['control_mean']:.2f}, n = {result['control_n']}")
        print(f"  Experimental Group: M = {result['experimental_mean']:.2f}, n = {result['experimental_n']}")
        print(f"  Mann-Whitney-U:   U = {stat:.3f}, p = {p_value:.3f}")
        print(f"  Cohen's d:        d = {cohens_d:.3f}")
        
        if p_value < 0.05:
            direction = "höher" if result['control_mean'] > result['experimental_mean'] else "niedriger"
            print(f"  → SIGNIFIKANT: Control Group ist {direction} (p < 0.05)")
        else:
            print(f"  → NICHT SIGNIFIKANT: Keine Unterschiede (p ≥ 0.05)")
    else:
        print(f"  → Nicht genügend Daten für Vergleich verfügbar")

# Zusammenfassung aller Vergleiche
print(f"\nZUSAMMENFASSUNG ALLER VERGLEICHE:")
print("=" * 40)
significant_count = sum(1 for r in comparison_results if r['significant'])
print(f"Signifikante Unterschiede: {significant_count}/{len(comparison_results)} Kategorien")

if significant_count > 0:
    print(f"\nSignifikante Kategorien:")
    for result in comparison_results:
        if result['significant']:
            direction = "höher" if result['control_mean'] > result['experimental_mean'] else "niedriger"
            effect_size = "groß" if abs(result['cohens_d']) > 0.8 else "mittel" if abs(result['cohens_d']) > 0.5 else "klein"
            print(f"  • {result['category']}: Control Group ist {direction} (p = {result['p_value']:.3f}, d = {result['cohens_d']:.3f}, {effect_size})")



STATISTISCHE VERGLEICHE ZWISCHEN GRUPPEN:
--------------------------------------------------

RELEVANCE & REALISM:
-------------------
  Control Group:    M = 4.00, n = 2
  Experimental Group: M = 4.15, n = 20
  Mann-Whitney-U:   U = 17.000, p = 0.755
  Cohen's d:        d = -0.212
  → NICHT SIGNIFIKANT: Keine Unterschiede (p ≥ 0.05)

CONFIDENCE IN PREDICTIONS:
-------------------------
  Control Group:    M = 3.00, n = 2
  Experimental Group: M = 4.05, n = 20
  Mann-Whitney-U:   U = 9.500, p = 0.208
  Cohen's d:        d = -1.523
  → NICHT SIGNIFIKANT: Keine Unterschiede (p ≥ 0.05)

USABILITY FOR RECRUITING:
------------------------
  Control Group:    M = 2.50, n = 2
  Experimental Group: M = 4.75, n = 20
  Mann-Whitney-U:   U = 2.500, p = 0.013
  Cohen's d:        d = -3.623
  → SIGNIFIKANT: Control Group ist niedriger (p < 0.05)

PERCEIVED VALUE & INTENTION TO USE:
----------------------------------
  Control Group:    M = 4.50, n = 2
  Experimental Group: M = 3.80, n = 20
  Mann-

In [10]:
# STATISTISCHE TESTS: NORMALITÄT UND GRUPPENVERGLEICHE
print("\nSTATISTISCHE TESTS")
print("=" * 30)

from scipy import stats
from scipy.stats import shapiro, kruskal, wilcoxon

# 1. NORMALITÄTSTESTS (Shapiro-Wilk) - GRUPPENSPEZIFISCH
print("\n1. NORMALITÄTSTESTS (Shapiro-Wilk) - GRUPPENSPEZIFISCH")
print("=" * 65)

# Control Group Normalitätstests
print("\nCONTROL GROUP NORMALITÄTSTESTS:")
print("-" * 40)
control_normality_results = []
for i, question in enumerate(control_group_questions):
    col = f'rating_{i+1}'
    if col in control_df.columns:
        ratings = control_df[col].dropna()
        if len(ratings) >= 3:  # Mindestens 3 Werte für Shapiro-Wilk
            stat, p_value = shapiro(ratings)
            control_normality_results.append({
                'Frage': f"Q{i+1}",
                'N': len(ratings),
                'Shapiro-W': round(stat, 4),
                'p-Wert': round(p_value, 4),
                'Normalverteilt': 'Ja' if p_value > 0.05 else 'Nein'
            })

if control_normality_results:
    control_normality_df = pd.DataFrame(control_normality_results)
    print(control_normality_df.to_string(index=False))
else:
    print("Keine Control Group Daten verfügbar")

# Experimental Group Normalitätstests
print("\nEXPERIMENTAL GROUP NORMALITÄTSTESTS:")
print("-" * 45)
experimental_normality_results = []
for i, question in enumerate(experimental_group_questions):
    col = f'rating_{i+1}'
    if col in experimental_df.columns:
        ratings = experimental_df[col].dropna()
        if len(ratings) >= 3:  # Mindestens 3 Werte für Shapiro-Wilk
            stat, p_value = shapiro(ratings)
            experimental_normality_results.append({
                'Frage': f"Q{i+1}",
                'N': len(ratings),
                'Shapiro-W': round(stat, 4),
                'p-Wert': round(p_value, 4),
                'Normalverteilt': 'Ja' if p_value > 0.05 else 'Nein'
            })

if experimental_normality_results:
    experimental_normality_df = pd.DataFrame(experimental_normality_results)
    print(experimental_normality_df.to_string(index=False))
else:
    print("Keine Experimental Group Daten verfügbar")

# Gesamtinterpretation
all_normality_results = control_normality_results + experimental_normality_results
non_normal_count = sum(1 for result in all_normality_results if result['Normalverteilt'] == 'Nein')
print(f"\nGESAMTINTERPRETATION: {non_normal_count}/{len(all_normality_results)} Fragen sind nicht normalverteilt")
print("→ Nicht-parametrische Tests (wie der Kruskal-Wallis) sind angemessen")



STATISTISCHE TESTS

1. NORMALITÄTSTESTS (Shapiro-Wilk) - GRUPPENSPEZIFISCH

CONTROL GROUP NORMALITÄTSTESTS:
----------------------------------------
Keine Control Group Daten verfügbar

EXPERIMENTAL GROUP NORMALITÄTSTESTS:
---------------------------------------------
Frage  N  Shapiro-W  p-Wert Normalverteilt
   Q1 10     0.8022  0.0154           Nein
   Q2 10     0.6405  0.0002           Nein
   Q3 10     0.8325  0.0359           Nein
   Q4 10     0.7516  0.0037           Nein
   Q5 10     0.8197  0.0251           Nein
   Q6 10     0.5093  0.0000           Nein
   Q7 10     0.5316  0.0000           Nein
   Q8 10     0.8148  0.0219           Nein
   Q9 10     0.9108  0.2869             Ja
  Q10 10     0.9240  0.3915             Ja
  Q11 10     0.6553  0.0003           Nein

GESAMTINTERPRETATION: 9/11 Fragen sind nicht normalverteilt
→ Nicht-parametrische Tests (wie der Kruskal-Wallis) sind angemessen


## Kruskal Walis Test

In [7]:
from scipy.stats import kruskal
import numpy as np

def run_kw_test(df, question_list, label, alpha=0.05):
    print(f"\n{label.upper()} KRUSKAL-WALLIS-TEST")
    print("=" * (len(label)+22))
    print("Testet, ob es signifikante Unterschiede zwischen den Fragen gibt")

    groups = []
    names = []

    for i, q in enumerate(question_list):
        col = f"rating_{i+1}"
        if col in df.columns:
            vals = df[col].dropna().values
            if len(vals) > 0:
                groups.append(vals)
                names.append(f"Q{i+1}")

    if len(groups) < 2:
        print("Nicht genügend Daten (mind. 2 Fragen mit Werten).")
        return

    # Kruskal-Wallis-Test
    h_stat, p_value = kruskal(*groups)

    print(f"H-Statistik: {h_stat:.4f}")
    print(f"p-Wert: {p_value:.4f}")

    if p_value < alpha:
        print(f"✓ SIGNIFIKANT: Unterschiede zwischen den Fragen (p < {alpha})")
    else:
        print(f"✗ NICHT SIGNIFIKANT: Keine Unterschiede (p ≥ {alpha})")

    # Effektstärke ε² (für KW)
    n_total = sum(len(g) for g in groups)
    k = len(groups)
    eps2 = (h_stat - (k - 1)) / (n_total - k) if (n_total - k) > 0 else np.nan
    print(f"Effektstärke (ε²): {eps2:.4f}")


# Beispiel-Aufrufe:
run_kw_test(control_df, control_group_questions, "Control Group")
run_kw_test(experimental_df, experimental_group_questions, "Experimental Group")


CONTROL GROUP KRUSKAL-WALLIS-TEST
Testet, ob es signifikante Unterschiede zwischen den Fragen gibt
H-Statistik: 7.0000
p-Wert: 0.4289
✗ NICHT SIGNIFIKANT: Keine Unterschiede (p ≥ 0.05)
Effektstärke (ε²): nan

EXPERIMENTAL GROUP KRUSKAL-WALLIS-TEST
Testet, ob es signifikante Unterschiede zwischen den Fragen gibt
H-Statistik: 34.2245
p-Wert: 0.0002
✓ SIGNIFIKANT: Unterschiede zwischen den Fragen (p < 0.05)
Effektstärke (ε²): 0.2447


In [8]:
import scikit_posthocs as sp
import pandas as pd
import numpy as np

def run_dunn_for_group(df, question_texts, group_label, comparable_questions, side_key, alpha=0.05):
    data_vals, data_labels = [], []
    for i, q in enumerate(question_texts):
        col = f"rating_{i+1}"
        if col in df.columns:
            vals = df[col].dropna().values
            if len(vals) > 0:
                data_vals.extend(vals)
                data_labels.extend([f"Q{i+1}"] * len(vals))

    if len(set(data_labels)) < 2:
        print(f"\n{group_label.upper()} – zu wenige Fragen mit Daten (mind. 2).")
        return

    long_df = pd.DataFrame({"rating": data_vals, "question": data_labels})
    posthoc = sp.posthoc_dunn(long_df, val_col='rating', group_col='question', p_adjust='holm')

    q_to_text = {f"Q{i+1}": txt for i, txt in enumerate(question_texts)}
    q_to_cat = {}
    for cat, sides in comparable_questions.items():
        for qnum in sides.get(side_key, []):
            q_to_cat[f"Q{qnum}"] = cat

    m = posthoc.copy()
    m.index.name = "q_i"
    m.columns.name = "q_j"
    long_mat = m.reset_index().melt(id_vars="q_i", var_name="q_j", value_name="p_adj")
    long_mat = long_mat[long_mat["q_i"] < long_mat["q_j"]].dropna(subset=["p_adj"])

    sig = long_mat[long_mat["p_adj"] < alpha].sort_values("p_adj").reset_index(drop=True)
    sig["q_i_text"] = sig["q_i"].map(q_to_text)
    sig["q_j_text"] = sig["q_j"].map(q_to_text)
    sig["cat_i"] = sig["q_i"].map(q_to_cat)
    sig["cat_j"] = sig["q_j"].map(q_to_cat)

    # 5) Ausgabe
    print(f"\nPOST-HOC DUNN-TEST ({group_label}) – signifikante Paare (Holm-korrigiert, alpha={alpha})")
    print("=" * (len(group_label) + 59))
    if sig.empty:
        print("Keine signifikanten Unterschiede.")
    else:
        for _, row in sig.iterrows():
            ci = f"[{row['cat_i']}]" if pd.notna(row['cat_i']) else ""
            cj = f"[{row['cat_j']}]" if pd.notna(row['cat_j']) else ""
            print(f"- {row['q_i']} {ci}  vs.  {row['q_j']} {cj}  | p_adj = {row['p_adj']:.4f}")
            print(f"    {row['q_i']}: {row['q_i_text']}")
            print(f"    {row['q_j']}: {row['q_j_text']}\n")

    print("\nKomplette p-Wert-Matrix (Holm-korrigiert):")
    print(posthoc.round(4))

run_dunn_for_group(
    df=control_df,
    question_texts=control_group_questions,
    group_label="Control Group",
    comparable_questions=comparable_questions,
    side_key="control",
    alpha=0.05
)

run_dunn_for_group(
    df=experimental_df,
    question_texts=experimental_group_questions,
    group_label="Experimental Group",
    comparable_questions=comparable_questions,
    side_key="experimental",
    alpha=0.05
)


POST-HOC DUNN-TEST (Control Group) – signifikante Paare (Holm-korrigiert, alpha=0.05)
Keine signifikanten Unterschiede.

Komplette p-Wert-Matrix (Holm-korrigiert):
     Q1   Q2   Q3   Q4      Q5   Q6      Q7   Q8
Q1  1.0  1.0  1.0  1.0  1.0000  1.0  1.0000  1.0
Q2  1.0  1.0  1.0  1.0  1.0000  1.0  1.0000  1.0
Q3  1.0  1.0  1.0  1.0  1.0000  1.0  1.0000  1.0
Q4  1.0  1.0  1.0  1.0  1.0000  1.0  1.0000  1.0
Q5  1.0  1.0  1.0  1.0  1.0000  1.0  0.5771  1.0
Q6  1.0  1.0  1.0  1.0  1.0000  1.0  1.0000  1.0
Q7  1.0  1.0  1.0  1.0  0.5771  1.0  1.0000  1.0
Q8  1.0  1.0  1.0  1.0  1.0000  1.0  1.0000  1.0

POST-HOC DUNN-TEST (Experimental Group) – signifikante Paare (Holm-korrigiert, alpha=0.05)
- Q10 [Perceived Value & Intention to Use]  vs.  Q6 [Usability for Recruiting]  | p_adj = 0.0054
    Q10: I could imagine integrating such a system with explanations into my daily recruiting workflow.
    Q6: The combination of predictions and explanations was straightforward and clear to understand.


## Wilcoxon-Tests

In [11]:
import numpy as np
import pandas as pd
from itertools import combinations
from scipy.stats import wilcoxon

def holm_correction(pvals_dict):
    items = sorted(pvals_dict.items(), key=lambda kv: kv[1]) 
    m = len(items)
    adj = {}
    running_max = 0.0
    for rank, ((i, j), p) in enumerate(items, start=1):
        padj = (m - rank + 1) * p
        padj = min(1.0, padj)
        running_max = max(running_max, padj)
        adj[(i, j)] = running_max
    return adj

def pairwise_wilcoxon(df_wide, q_labels, min_pairs=5):
    p_raw = {}
    n_pairs = {}
    for i, j in combinations(q_labels, 2):
        s1 = df_wide[i]
        s2 = df_wide[j]
        mask = s1.notna() & s2.notna()
        x = s1[mask].to_numpy()
        y = s2[mask].to_numpy()
        n = len(x)
        n_pairs[(i, j)] = n
        if n < min_pairs:
            p_raw[(i, j)] = np.nan
            continue
        try:
            stat, p = wilcoxon(x, y, zero_method="wilcox", alternative="two-sided", correction=False, method="auto")
        except ValueError:
            p = 1.0
        p_raw[(i, j)] = p
    return p_raw, n_pairs

def run_wilcoxon_for_group(df, question_texts, group_label, comparable_questions, side_key, alpha=0.05, min_pairs=5):

    cols = {}
    for i, _ in enumerate(question_texts):
        col_raw = f"rating_{i+1}"
        if col_raw in df.columns:
            cols[f"Q{i+1}"] = df[col_raw]
    if len(cols) < 2:
        print(f"\n{group_label.upper()} – zu wenige Fragen (mind. 2 Spalten vorhanden).")
        return

    wide = pd.DataFrame(cols)

    q_to_text = {f"Q{i+1}": txt for i, txt in enumerate(question_texts)}
    q_to_cat = {}
    for cat, sides in comparable_questions.items():
        for qnum in sides.get(side_key, []):
            q_to_cat[f"Q{qnum}"] = cat

    q_labels = list(wide.columns)

    p_raw, n_pairs = pairwise_wilcoxon(wide, q_labels, min_pairs=min_pairs)
    p_raw_valid = {k: v for k, v in p_raw.items() if not np.isnan(v)}
    if len(p_raw_valid) == 0:
        print(f"\n{group_label.upper()} – keine auswertbaren Paare (min_pairs={min_pairs}).")
        return

    p_adj = holm_correction(p_raw_valid)

    all_labels = q_labels
    mat_raw = pd.DataFrame(1.0, index=all_labels, columns=all_labels, dtype=float)
    mat_adj = pd.DataFrame(1.0, index=all_labels, columns=all_labels, dtype=float)
    for (i, j), p in p_raw.items():
        mat_raw.loc[i, j] = mat_raw.loc[j, i] = p if not np.isnan(p) else np.nan
    for (i, j), p in p_adj.items():
        mat_adj.loc[i, j] = mat_adj.loc[j, i] = p

    sig_rows = []
    for (i, j), p in p_adj.items():
        if p < alpha:
            sig_rows.append({
                "q_i": i, "q_j": j, "p_adj": p,
                "n_pairs": n_pairs[(i, j)],
                "cat_i": q_to_cat.get(i, np.nan),
                "cat_j": q_to_cat.get(j, np.nan),
                "q_i_text": q_to_text.get(i, i),
                "q_j_text": q_to_text.get(j, j),
            })
    sig_df = pd.DataFrame(sig_rows).sort_values("p_adj") if sig_rows else pd.DataFrame(columns=[
        "q_i","q_j","p_adj","n_pairs","cat_i","cat_j","q_i_text","q_j_text"
    ])

    # Ausgabe
    print(f"\nWILCOXON SIGNED-RANK – PAARVERGLEICHE ({group_label}) (Holm-korrigiert, alpha={alpha}, min_pairs={min_pairs})")
    print("=" * (len(group_label) + 61))
    if sig_df.empty:
        print("Keine signifikanten Unterschiede.")
    else:
        for _, row in sig_df.iterrows():
            ci = f"[{row['cat_i']}]" if pd.notna(row['cat_i']) else ""
            cj = f"[{row['cat_j']}]" if pd.notna(row['cat_j']) else ""
            print(f"- {row['q_i']} {ci}  vs.  {row['q_j']} {cj}  | p_adj = {row['p_adj']:.4f} | n={int(row['n_pairs'])}")
            print(f"    {row['q_i']}: {row['q_i_text']}")
            print(f"    {row['q_j']}: {row['q_j_text']}\n")

    print("\nKomplette p-Wert-Matrix (roh):")
    print(mat_raw.round(4))
    print("\nKomplette p-Wert-Matrix (Holm-korrigiert):")
    print(mat_adj.round(4))

run_wilcoxon_for_group(
    df=control_df,
    question_texts=control_group_questions,
    group_label="Control Group",
    comparable_questions=comparable_questions,
    side_key="control",
    alpha=0.05,
    min_pairs=2
)

run_wilcoxon_for_group(
    df=experimental_df,
    question_texts=experimental_group_questions,
    group_label="Experimental Group",
    comparable_questions=comparable_questions,
    side_key="experimental",
    alpha=0.05,
    min_pairs=2
)


CONTROL GROUP – keine auswertbaren Paare (min_pairs=2).

WILCOXON SIGNED-RANK – PAARVERGLEICHE (Experimental Group) (Holm-korrigiert, alpha=0.05, min_pairs=2)
Keine signifikanten Unterschiede.

Komplette p-Wert-Matrix (roh):
         Q1      Q2      Q3      Q4      Q5      Q6      Q7      Q8      Q9  \
Q1   1.0000  0.0209  0.4795  0.4795  0.0588  0.0094  0.0152  0.1797  1.0000   
Q2   0.0209  1.0000  0.0196  0.0196  0.1573  0.3173  0.6547  0.0578  0.0488   
Q3   0.4795  0.0196  1.0000  1.0000  0.3340  0.0304  0.0461  0.6547  0.6078   
Q4   0.4795  0.0196  1.0000  1.0000  0.2568  0.0209  0.0335  0.7055  0.6078   
Q5   0.0588  0.1573  0.3340  0.2568  1.0000  0.0836  0.1597  0.5271  0.1317   
Q6   0.0094  0.3173  0.0304  0.0209  0.0836  1.0000  0.5637  0.0209  0.0158   
Q7   0.0152  0.6547  0.0461  0.0335  0.1597  0.5637  1.0000  0.0196  0.0141   
Q8   0.1797  0.0578  0.6547  0.7055  0.5271  0.0209  0.0196  1.0000  0.3173   
Q9   1.0000  0.0488  0.6078  0.6078  0.1317  0.0158  0.0141  0.

In [12]:
import numpy as np
import pandas as pd
from itertools import combinations
from scipy.stats import wilcoxon

# -------------------------------
# 1) Holm-Korrektur
# -------------------------------
def holm_correction(pvals_dict):
    items = sorted(pvals_dict.items(), key=lambda kv: kv[1])  # aufsteigend nach p
    m = len(items)
    adj = {}
    running_max = 0.0
    for rank, (key, p) in enumerate(items, start=1):
        padj = min(1.0, (m - rank + 1) * p)
        running_max = max(running_max, padj)  # step-up
        adj[key] = running_max
    return adj

# -------------------------------
# 2) Kategorie-Scores bauen
# -------------------------------
def build_category_scores(df, question_texts, comparable_questions, side_key, agg="mean"):
    """
    df: DataFrame mit Spalten rating_1 ... rating_k
    question_texts: Liste der Fragetexte (Länge k)
    comparable_questions: Dict Deiner Kategorien
    side_key: "control" oder "experimental"
    agg: "mean" oder "median"
    return: DataFrame mit Spalten = Kategorienamen, Zeilen = Personen
    """
    # Map Q-Label -> Spaltenname im DF
    q_to_col = {f"Q{i+1}": f"rating_{i+1}" for i in range(len(question_texts))}

    cat_frames = {}
    for cat, sides in comparable_questions.items():
        idxs = sides.get(side_key, [])
        qs = [f"Q{q}" for q in idxs]
        cols = [q_to_col[q] for q in qs if q_to_col[q] in df.columns]
        if not cols:
            continue
        block = df[cols]
        if agg == "mean":
            cat_frames[cat] = block.mean(axis=1)
        elif agg == "median":
            cat_frames[cat] = block.median(axis=1)
        else:
            raise ValueError("agg muss 'mean' oder 'median' sein.")
    if not cat_frames:
        return pd.DataFrame()
    return pd.DataFrame(cat_frames)

# -------------------------------
# 3) Paarweiser Wilcoxon auf Kategorien
# -------------------------------
def pairwise_wilcoxon_categories(cat_df, min_pairs=5):
    """
    cat_df: DataFrame mit Kategorien als Spalten
    return:
      p_raw: {(c1,c2): p}
      stats: {(c1,c2): {"n": n, "W": stat, "r_rb": r}}
    """
    cats = list(cat_df.columns)
    p_raw, stats = {}, {}
    for c1, c2 in combinations(cats, 2):
        x = cat_df[c1]
        y = cat_df[c2]
        mask = x.notna() & y.notna()
        x_, y_ = x[mask].to_numpy(), y[mask].to_numpy()
        n = len(x_)
        if n < min_pairs:
            continue
        try:
            # zweiseitig; W = Summe der positiven Ränge
            W, p = wilcoxon(x_, y_, zero_method="wilcox", alternative="two-sided",
                            correction=False, method="auto")
        except ValueError:
            # z.B. alle Differenzen genau 0 -> kein Test möglich
            continue
        p_raw[(c1, c2)] = float(p)
        # Rank-biserial correlation (für gepaarte Wilcoxon):
        # r_rb = 1 - (2*W) / (n*(n+1))
        r_rb = 1.0 - (2.0 * W) / (n * (n + 1))
        stats[(c1, c2)] = {"n": n, "W": float(W), "r_rb": float(r_rb)}
    return p_raw, stats

# -------------------------------
# 4) Hauptfunktion für eine Gruppe
# -------------------------------
def run_wilcoxon_categories(df, question_texts, group_label,
                            comparable_questions, side_key,
                            alpha=0.05, min_pairs=5, agg="mean"):
    """
    Führt Wilcoxon-Tests zwischen Kategorie-Scores (gepairt) durch.
    Druckt signifikante Paare + komplette p-Matrix (adj).
    """
    cat_df = build_category_scores(df, question_texts, comparable_questions, side_key, agg=agg)
    if cat_df.empty or cat_df.shape[1] < 2:
        print(f"\n{group_label.upper()} – zu wenige Kategorien für Vergleiche.")
        return

    p_raw, stats = pairwise_wilcoxon_categories(cat_df, min_pairs=min_pairs)
    if not p_raw:
        print(f"\n{group_label.upper()} – keine auswertbaren Kategorie-Paare (min_pairs={min_pairs}).")
        return

    # Holm-Korrektur
    p_adj = holm_correction(p_raw)

    # Kategorie-Liste & p-Matrix (adj)
    cats = list(cat_df.columns)
    mat_adj = pd.DataFrame(1.0, index=cats, columns=cats, dtype=float)
    for (c1, c2), p in p_adj.items():
        mat_adj.loc[c1, c2] = mat_adj.loc[c2, c1] = p

    # Signifikante Paare sammeln
    rows = []
    for (c1, c2), p in p_adj.items():
        if p < alpha:
            info = stats.get((c1, c2), {})
            rows.append({
                "cat_1": c1, "cat_2": c2,
                "p_adj": p,
                "n_pairs": info.get("n", np.nan),
                "W": info.get("W", np.nan),
                "r_rb": info.get("r_rb", np.nan)
            })
    sig = pd.DataFrame(rows).sort_values("p_adj") if rows else pd.DataFrame(columns=["cat_1","cat_2","p_adj","n_pairs","W","r_rb"])

    # Ausgabe
    print(f"\nWILCOXON – KATEGORIEVERGLEICHE ({group_label}) (agg={agg}, Holm, alpha={alpha}, min_pairs={min_pairs})")
    print("=" * (len(group_label) + 61))
    if sig.empty:
        print("Keine signifikanten Unterschiede zwischen Kategorien.")
    else:
        for _, r in sig.iterrows():
            print(f"- {r['cat_1']}  vs.  {r['cat_2']}  | p_adj = {r['p_adj']:.4f} | n={int(r['n_pairs'])} | W={r['W']:.1f} | r_rb={r['r_rb']:.3f}")

    print("\nKomplette p-Wert-Matrix (Holm-korrigiert):")
    print(mat_adj.round(4))

# -------------------------------
# 5) Aufrufe für beide Gruppen
# -------------------------------
run_wilcoxon_categories(
    df=control_df,
    question_texts=control_group_questions,
    group_label="Control Group",
    comparable_questions=comparable_questions,
    side_key="control",
    alpha=0.05,
    min_pairs=5,   # Du kannst auf 3 senken, wenn Daten knapp sind
    agg="mean"     # oder "median"
)

run_wilcoxon_categories(
    df=experimental_df,
    question_texts=experimental_group_questions,
    group_label="Experimental Group",
    comparable_questions=comparable_questions,
    side_key="experimental",
    alpha=0.05,
    min_pairs=5,
    agg="mean"
)


CONTROL GROUP – keine auswertbaren Kategorie-Paare (min_pairs=5).

WILCOXON – KATEGORIEVERGLEICHE (Experimental Group) (agg=mean, Holm, alpha=0.05, min_pairs=5)
Keine signifikanten Unterschiede zwischen Kategorien.

Komplette p-Wert-Matrix (Holm-korrigiert):
                                    Relevance & Realism  \
Relevance & Realism                              1.0000   
Confidence in Predictions                        0.9844   
Usability for Recruiting                         0.0981   
Perceived Value & Intention to Use               0.3619   

                                    Confidence in Predictions  \
Relevance & Realism                                    0.9844   
Confidence in Predictions                              1.0000   
Usability for Recruiting                               0.1392   
Perceived Value & Intention to Use                     0.9844   

                                    Usability for Recruiting  \
Relevance & Realism                                   